# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
path ='/Users/Justin/Documents/Bootcamp_Home folder/homework/Module 6/python-api-challenge'

city_data_df = pd.read_csv(path +"/output_data/cities.csv")

#Convert Date time from Unix to local time
city_data_df["Date"]=(pd.to_datetime(city_data_df["Date"],unit='s'))

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,79.02,72,5,14.47,PF,2023-02-01 05:24:47
1,1,kristiansund,63.1115,7.7320,28.45,80,100,3.44,NO,2023-02-01 05:24:48
2,2,barrow,71.2906,-156.7887,-5.78,77,0,25.32,US,2023-02-01 05:22:20
3,3,sistranda,63.7256,8.8340,26.56,69,100,11.45,NO,2023-02-01 05:24:49
4,4,puerto ayora,-0.7393,-90.3518,75.54,92,73,4.88,EC,2023-02-01 05:24:50


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_df = city_data_df

# A max temperature lower than 80 degrees but higher than 70.
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] < 80) & (ideal_weather_df["Max Temp"] > 70)]

# Wind speed less than 5 mph.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]

#Humidity between 70% - 80%
#ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Humidity"] < 70) & (ideal_weather_df["Humidity"] > 30)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()


# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
91,91,willowmore,-33.2926,23.4895,72.63,60,0,1.48,ZA,2023-02-01 05:25:48
124,124,cidreira,-30.1811,-50.2056,74.23,92,0,3.06,BR,2023-02-01 05:26:10
227,227,east london,-33.0153,27.9116,74.34,83,0,5.75,ZA,2023-02-01 05:22:24
323,323,sweetwater,25.7634,-80.3731,75.99,89,0,5.75,US,2023-02-01 05:28:21
333,333,canelones,-34.6444,-56.2886,73.49,90,0,6.29,UY,2023-02-01 05:28:28


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

hotel_df

,City,Country,Lat,Lng,Humidity
91,willowmore,ZA,-33.2926,23.4895,60
124,cidreira,BR,-30.1811,-50.2056,92
227,east london,ZA,-33.0153,27.9116,83
323,sweetwater,US,25.7634,-80.3731,89
333,canelones,UY,-34.6444,-56.2886,90
347,amravati,IN,20.9333,77.7500,37
388,tlatenchi,MX,18.5833,-99.2000,40
524,ixtapa,MX,20.7000,-105.2000,94


In [15]:
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel Name", "") 

#Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
91,willowmore,ZA,-33.2926,23.4895,60,
124,cidreira,BR,-30.1811,-50.2056,92,
227,east london,ZA,-33.0153,27.9116,83,
323,sweetwater,US,25.7634,-80.3731,89,
333,canelones,UY,-34.6444,-56.2886,90,
347,amravati,IN,20.9333,77.7500,37,
388,tlatenchi,MX,18.5833,-99.2000,40,
524,ixtapa,MX,20.7000,-105.2000,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000

# set up a parameters dictionary
params = {"apiKey":geoapify_key,
         "categories": 'accommodation.hotel'}
         

# Print a message to follow up the hotel search
print("Starting hotel search")

Starting hotel search


In [17]:
%%capture --no-display

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    
    # Convert the API response to JSON format
    name_address = name_address.json()
  
    
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df
    


,City,Country,Lat,Lng,Humidity,Hotel Name
91,willowmore,ZA,-33.2926,23.4895,60,No hotel found
124,cidreira,BR,-30.1811,-50.2056,92,Hotel Castelo
227,east london,ZA,-33.0153,27.9116,83,No hotel found
323,sweetwater,US,25.7634,-80.3731,89,Courtyard by Marriott Miami at Dolphin Mall
333,canelones,UY,-34.6444,-56.2886,90,No hotel found
347,amravati,IN,20.9333,77.7500,37,Hotel Hindustan International
388,tlatenchi,MX,18.5833,-99.2000,40,MonteCarlo
524,ixtapa,MX,20.7000,-105.2000,94,Ma. Cristina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot_3 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols= ["Hotel Name", "Country"], 
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_3

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)